In [2]:
import pandas as pd

In [23]:
datasets = {
    'messages': pd.read_csv('data/apparel-messages.csv'),
    'purchases': pd.read_csv('data/apparel-purchases.csv'),
    'target_binary': pd.read_csv('data/apparel-target_binary.csv'),
    'daily_event': pd.read_csv('data/full_campaign_daily_event.csv'),
    'daily_event_chan': pd.read_csv('data/full_campaign_daily_event_channel.csv')
}

In [40]:
for f in datasets:
    print('---'*20)
    print(f.upper())
    print(datasets[f].head(3).T)
    print('\nShape\n')
    print(datasets[f].shape)
    print('\nInfo\n')
    print(datasets[f].info())
    print('\nDescribe\n')
    print(datasets[f].columns)
    print('\n')
    print(datasets[f].describe())
    print('\nNulls\n')
    print(datasets[f].isnull().sum())
    print('\nUnique\n')
    print(datasets[f].nunique())
    print('---'*20)

------------------------------------------------------------
MESSAGES
                                                       0  \
bulk_campaign_id                                    4439   
client_id                            1515915625626736623   
message_id        1515915625626736623-4439-6283415ac07ea   
event                                               open   
channel                                            email   
date                                          2022-05-19   
created_at                           2022-05-19 00:14:20   

                                                       1  \
bulk_campaign_id                                    4439   
client_id                            1515915625490086521   
message_id        1515915625490086521-4439-62834150016dd   
event                                               open   
channel                                            email   
date                                          2022-05-19   
created_at                   